In [93]:
import pandas as pd
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [94]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [95]:
###Drop Nan Values
df=df.dropna()

In [96]:
## Get the Independent Features

X=df.drop('label',axis=1)

## Get the Dependent features
y=df['label']

In [97]:
X.shape

(18285, 4)

In [98]:
y.shape

(18285,)

In [99]:
import tensorflow as tf

In [100]:
tf.__version__

'2.15.0'

In [101]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [102]:
### Vocabulary size
voc_size=6000

In [103]:
messages=X.copy()
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [104]:
import nltk
import re
from nltk.corpus import stopwords

In [105]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [106]:
messages.reset_index(inplace=True)
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [107]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [108]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [109]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [110]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3391, 27, 715, 3459, 2765, 5192, 1552, 5303, 4527, 1365],
 [4405, 2455, 5354, 5798, 644, 351, 4356],
 [761, 5489, 1991, 4129],
 [3594, 3347, 2500, 2974, 4933, 2318],
 [3679, 644, 3057, 1162, 5013, 1409, 644, 755, 4294, 4412],
 [5660,
  2980,
  4240,
  1416,
  338,
  1019,
  5324,
  1639,
  2463,
  4358,
  1540,
  934,
  2965,
  4109,
  4356],
 [905, 684, 3866, 726, 3325, 4952, 3644, 2321, 2200, 5361, 3065],
 [884, 808, 2398, 2669, 2987, 241, 1019, 1228, 2200, 5361, 3065],
 [5395, 2277, 3521, 1464, 4311, 1170, 4051, 1969, 1019, 3376],
 [4639, 4499, 4161, 4882, 4308, 2988, 2954, 5956],
 [1565, 4459, 2070, 792, 2684, 4627, 3161, 1487, 3696, 4293, 2058],
 [2974, 3304, 2765, 1170, 1019, 2987],
 [4568, 5173, 4601, 589, 5560, 4035, 5920, 3961, 5337],
 [5592, 3181, 2532, 1922, 4772, 636, 1966, 2200, 5361, 3065],
 [1481, 2552, 76, 3398, 3736, 2200, 5361, 3065],
 [5098, 4455, 2097, 250, 964, 1042, 4023, 618, 3604, 1784],
 [2305, 666, 2455],
 [415, 2587, 3307, 5100, 1019, 5494, 957, 4356],
 [22

In [111]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [112]:
onehot_repr[1]

[4405, 2455, 5354, 5798, 644, 351, 4356]

In [113]:
# Embedding Representation
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[3391   27  715 ...    0    0    0]
 [4405 2455 5354 ...    0    0    0]
 [ 761 5489 1991 ...    0    0    0]
 ...
 [ 842 5287  112 ...    0    0    0]
 [2888 2987 4202 ...    0    0    0]
 [5987 3208  432 ...    0    0    0]]


In [114]:
embedded_docs[0]

array([3391,   27,  715, 3459, 2765, 5192, 1552, 5303, 4527, 1365,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [ ]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(Dropout(0.3))
model.add(LSTM(100))
# model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 20, 40)            240000    
                                                                 
 lstm_5 (LSTM)               (None, 100)               56400     
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 296501 (1.13 MB)
Trainable params: 296501 (1.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [116]:
len(embedded_docs),y.shape

(18285, (18285,))

In [117]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [118]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [119]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33)

In [120]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 15s 52ms/step - loss: 0.3100 - accuracy: 0.8458 - val_loss: 0.1874 - val_accuracy: 0.9248
Epoch 2/10
192/192 [==============================] - 10s 51ms/step - loss: 0.1386 - accuracy: 0.9464 - val_loss: 0.1859 - val_accuracy: 0.9253
Epoch 3/10
192/192 [==============================] - 8s 44ms/step - loss: 0.0948 - accuracy: 0.9660 - val_loss: 0.2270 - val_accuracy: 0.9190
Epoch 4/10
192/192 [==============================] - 9s 46ms/step - loss: 0.0634 - accuracy: 0.9793 - val_loss: 0.2950 - val_accuracy: 0.9228
Epoch 5/10
192/192 [==============================] - 9s 45ms/step - loss: 0.0480 - accuracy: 0.9830 - val_loss: 0.3311 - val_accuracy: 0.9206
Epoch 6/10
192/192 [==============================] - 9s 46ms/step - loss: 0.0342 - accuracy: 0.9890 - val_loss: 0.3554 - val_accuracy: 0.9110
Epoch 7/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0235 - accuracy: 0.9923 - val_loss: 0.4086 - val_accuracy: 0.91

In [122]:
X_test

array([[1725, 5186,   27, ...,    0,    0,    0],
       [3054, 3601, 3557, ...,    0,    0,    0],
       [1582, 4963, 2737, ...,    0,    0,    0],
       ...,
       [1025, 5156, 3295, ...,    0,    0,    0],
       [5836, 3621, 4104, ...,    0,    0,    0],
       [4109, 4079,  386, ...,    0,    0,    0]])

In [123]:
y_pred=model.predict(X_test)

189/189 [==============================] - 4s 10ms/step


In [124]:
# with the help of AUC ROC Curve, we can set the threshold
y_pred=np.where(y_pred > 0.6, 1,0) 

In [125]:
from sklearn.metrics import confusion_matrix

In [126]:
confusion_matrix(y_test,y_pred)

array([[3229,  243],
       [ 265, 2298]], dtype=int64)

In [127]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.915824357912179

In [128]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.93      0.93      3472
           1       0.90      0.90      0.90      2563

    accuracy                           0.92      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.92      0.92      0.92      6035

